In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [2]:
print('current working directory: {}'.format(os.getcwd()))

current working directory: C:\Users\agarw\Dropbox\Kaggle data-science-bowl 2019\Prarit-data-science-bowl-2019


In [3]:
print('files in cwd: {}'.format(os.listdir()))

files in cwd: ['.git', '.ipynb_checkpoints', 'Feature Engineering Speed Up.ipynb', 'Feature Engineering.ipynb', 'Initial EDA.ipynb', 'sample_submission.csv', 'specs.csv', 'test.csv', 'train.csv', 'train_features.csv', 'train_labels.csv']


In [4]:
trainpath='train.csv'
train=pd.read_csv(trainpath)

In [5]:
trainlblspath='train_labels.csv'
trainlbls=pd.read_csv(trainlblspath)

In [6]:
print('shape of train: {}'.format(train.shape))
print('shape of trainlbls: {}'.format(trainlbls.shape))

shape of train: (11341042, 11)
shape of trainlbls: (17690, 7)


In [7]:
n_players=train.installation_id.nunique()
print('number of unique installation ids: {}'.format(n_players))

number of unique installation ids: 17000


In [8]:
train.head(2)

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world
0,27253bdc,45bb1e1b6b50c07b,2019-09-06T17:53:46.937Z,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE
1,27253bdc,17eeb7f223665f53,2019-09-06T17:54:17.519Z,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Magma Peak - Level 1,Clip,MAGMAPEAK


In [9]:
trainlbls.head(2)

,game_session,installation_id,title,num_correct,num_incorrect,accuracy,accuracy_group
0,6bdf9623adc94d89,0006a69f,Mushroom Sorter (Assessment),1,0,1.0,3
1,77b8ee947eb84b4e,0006a69f,Bird Measurer (Assessment),0,11,0.0,0


In [10]:
# converting timestamp in train to datetime
train['datetime']=pd.to_datetime(train.timestamp)
train.head(2)

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world,datetime
0,27253bdc,45bb1e1b6b50c07b,2019-09-06T17:53:46.937Z,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE,2019-09-06 17:53:46.937
1,27253bdc,17eeb7f223665f53,2019-09-06T17:54:17.519Z,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Magma Peak - Level 1,Clip,MAGMAPEAK,2019-09-06 17:54:17.519


In [11]:
train.drop(columns=['timestamp'], inplace=True)
train.head(2)

,event_id,game_session,event_data,installation_id,event_count,event_code,game_time,title,type,world,datetime
0,27253bdc,45bb1e1b6b50c07b,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE,2019-09-06 17:53:46.937
1,27253bdc,17eeb7f223665f53,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Magma Peak - Level 1,Clip,MAGMAPEAK,2019-09-06 17:54:17.519


In [12]:
# number of unique sessions in train
print('The number of unique sessions in train are: {}'.format(train.game_session.nunique()))

The number of unique sessions in train are: 303319


In [13]:
# group the data according to installation_id, type, title and game_session
# Since we are only interested in things like, total time and number of events in each game session, we
# will most probably need all the rows in the train_data for now.
# so group them as above and then only choosing the relevant rows might help
# also notice that we don't care about sorting the game_sessions by their ids, so we can turn off the sort parameter which is True by default. This will improve the time 
trngrp=train.groupby(['game_session'], sort=False)

In [14]:
# note that the total time spent and the number of events in a particular game session can be easily obtained by looking at the last row in the data for each game_session
# Basically, the game_time column contains the number of milli-sec passed (since the start of the session) when the event occurred
# similarly, the entry under the event_count column for the last row will give us the number of events that occurred in that session
# Meanwhile the first row of each game_session gives us the start time

# confirm that the the first row of each game_session is indeed the begining of the session
# this implies that the event_count of that row should be 1
print('The first row in each group has event count == 1? :{}'.format((trngrp.first().event_count==1).all()))

# start of each game_session
start=trngrp.first()

# end of each game_session
end=trngrp.last()

The first row in each group has event count == 1? :True


In [15]:
# confirm that the rows in the datafram 'end' indeed correspond to the last event in that game_session
# this can be confirmed by checking that the entry under event_count of 'end' is indeed the largest event_count value for each session
print('All entries in the DataFrame {} are equal to max event_count for their game?: {}'.format('end',(end.event_count==trngrp.event_count.max()).all()))

All entries in the DataFrame end are equal to max event_count for their game?: False


In [16]:
# Apparently, there are rows in the dataframe 'end' which do not correspond to the max event_count for that session
# let's look at the corresponding sessions and try to understand why is this the case
end['max_event_count']=(end.event_count==trngrp.event_count.max())
not_max=end.loc[end.max_event_count==False]

# let's look at the first session in not_max
weird_session_id=not_max.index[0]
print('An example of game_session when the dataframe end does not have max event_count:{}'.format(weird_session_id))

An example of game_session when the dataframe end does not have max event_count:fa7e513faec3d0d8


In [17]:
# To access a particular group in a groupby object, use the 'get_group()' method
# this tip was given in the following stack-exchange post: https://stackoverflow.com/questions/22702486/pandas-how-to-get-a-particular-group-after-groupby

weird_session=trngrp.get_group(weird_session_id)

print('max event_count in {} is {}'.format(weird_session_id, weird_session.event_count.max()))
print('the corresponding event_count in the dataframe end is {}'.format(end.loc[weird_session_id, 'event_count']))

max event_count in fa7e513faec3d0d8 is 63
the corresponding event_count in the dataframe end is 62


In [18]:
weird_session

,datetime,event_code,event_count,event_data,event_id,game_time,installation_id,title,type,world
4752,2019-08-29 16:17:08.865,2000,1,"{""version"":""1.0"",""level"":0,""round"":0,""event_co...",6d90d394,0,0006a69f,Scrub-A-Dub,Game,MAGMAPEAK
4753,2019-08-29 16:17:11.207,4010,2,"{""coordinates"":{""x"":1142,""y"":951,""stage_width""...",7040c096,2342,0006a69f,Scrub-A-Dub,Game,MAGMAPEAK
4754,2019-08-29 16:17:13.073,2080,3,"{""movie_id"":""scrubadub_intro"",""duration"":13766...",5a848010,4209,0006a69f,Scrub-A-Dub,Game,MAGMAPEAK
4755,2019-08-29 16:17:14.657,2081,4,"{""movie_id"":""scrubadub_intro"",""duration"":13766...",c1cac9a2,5793,0006a69f,Scrub-A-Dub,Game,MAGMAPEAK
4756,2019-08-29 16:17:16.582,3010,7,"{""description"":""First you need to match the an...",f71c4741,7718,0006a69f,Scrub-A-Dub,Game,MAGMAPEAK
4757,2019-08-29 16:17:16.582,2020,6,"{""round_target"":{""size"":2,""type"":""Tub"",""animal...",26fd2d99,7718,0006a69f,Scrub-A-Dub,Game,MAGMAPEAK
4758,2019-08-29 16:17:16.582,2040,5,"{""level"":1,""round"":0,""event_count"":5,""game_tim...",dcaede90,7718,0006a69f,Scrub-A-Dub,Game,MAGMAPEAK
4759,2019-08-29 16:17:17.765,3110,8,"{""description"":""First you need to match the an...",f7e47413,8901,0006a69f,Scrub-A-Dub,Game,MAGMAPEAK
4760,2019-08-29 16:17:17.766,4020,9,"{""size"":2,""item_type"":""Tub"",""position"":1,""anim...",5c3d2b2f,8901,0006a69f,Scrub-A-Dub,Game,MAGMAPEAK
4761,2019-08-29 16:17:18.947,4070,10,"{""coordinates"":{""x"":1374,""y"":319,""stage_width""...",cf82af56,10085,0006a69f,Scrub-A-Dub,Game,MAGMAPEAK


by looking at all the events in game_session == '007881b97f50de0f', we realize that while for most of the game_sessions, the rows are sorted by their event_count, but for many sessions, this does not appear to be the case. Thus, picking up the last row in each game_session is not a reliable method to find the last event in that game_session. 

One way to circumvent this is to sort the rows of each group according to their event_count. A similar problem was discussed in [this](https://stackoverflow.com/questions/15705630/get-the-rows-which-have-the-max-value-in-groups-using-groupby) stackexchange post. This is very useful post and in particular the answers by user a) Zelazny7, b) landewednack and c) WeNYoBen look particularly useful and promising. I personally think the best answer is by WeNYoBen (though it does not have the most votes). Not only is it elegant, but someone has also commented that it is faster than the solutions by the other 2. WeNYoBen's solution follows the logic that we should first sort the dataframe itself by event_count and then groupby 'game_session'. In this way the rows in the resulting groupby object so created will automatically be ordered by their event_count.  We will follow this approach. 

In [40]:
trngrp=train.sort_values('event_count', axis=0).groupby('game_session', sort=False)
start=trngrp.first()
end=trngrp.last()

print('The first row in each group has event count == 1? :{}'.format((start.event_count==1).all()))
print('The last row in each group has maximum event count?: {}'.
      format((end.event_count==trngrp.event_count.max()).all()))

The first row in each group has event count == 1? :True
The last row in each group has maximum event count?: True


In [41]:
# for all the assessments, let us also pick out all the events which have code == 4100 or 4110
# Since these events correspond to attempts made by the player to solve the assessment
attempts=train.loc[(train.type=='Assessment')&((train.event_code==4100)| (train.event_code==4110))].groupby('game_session', sort=False)

In [24]:
attempts.ngroups

17692

Recall, some assessments that were unsuccessfully attempted but were abandoned before arriving at a correct solution. How do I find assessments that were abandoned before completion? I tried to look at the last event in some completed assessments but could not find something that seemed to be true for all completed assessments. 

For example, I looked at the game_session == '901acc108f55a5a1'. This was a completed assessment. The last event (row = 2232 of train) had an event_code = 2010, its event_data had a substring 'game_completed' and its event_id is 'a5be6304'.

In [26]:
end.loc['901acc108f55a5a1', :]

event_id                                                    a5be6304
event_data         {"session_duration":39803,"exit_type":"game_co...
installation_id                                             0006a69f
event_count                                                       48
event_code                                                      2010
game_time                                                      39803
title                                   Mushroom Sorter (Assessment)
type                                                      Assessment
world                                                    TREETOPCITY
datetime                                  2019-08-06 05:22:41.147000
Name: 901acc108f55a5a1, dtype: object

In [27]:
end.loc['901acc108f55a5a1'].event_data

'{"session_duration":39803,"exit_type":"game_completed","event_count":48,"game_time":39803,"event_code":2010}'

 By looking at the above result, we might think that all completed assessments might have 2010 for the event_code of their last event. Let's check if this is true.

In [34]:
print('No. of assessments containing an event with event code 2010: {}'.format(train.loc[(train.type=='Assessment') & (train.event_code==2010)].game_session.nunique()))

No. of assessments containing an event with event code 2010: 11691


Recall that train_labels.csv has data of 17690 different assessments. Assuming that these all were assessments that were not abandoned prematurely, we see that there are only 11691 assessments that have an event with event_code 2010. This is far less than the assessments in train_labels and hence can not be a universal approach to finding completed assessments. 

Similarly, we can look for assessments whose last event has the phrase 'game completed' in its event_data

In [35]:
# find the position of the phrase 'game completed' in event_data
# if the substring exists then str.find(substring) returns a non-negative value else it returns -1
completed=end.loc[end.type=='Assessment'].event_data.apply(lambda x: x.find('game_completed')>-1)
print('No. of assessments containing the phrase game_completed: {} '.format(completed.loc[completed==True].size))

No. of assessments containing the phrase game_completed: 2925 


Once again, we see that this number is way less than the number of sessions than recored in train_labels.csv

Let us check if all completed assessments will have event_id == 'a5be6304'. 

In [38]:
completed=end.loc[(end.type=='Assessment') &(end.event_id=='a5be6304')]
print('No. of assessments having event_id a5be6304: {}'.format(completed.shape[0]))

No. of assessments having event_id a5be6304: 2926


This too is way less than the number of sessions in train_labels.csv

As a last resort, we notice that the event_data for the last event of a completed session seems always contain info about session duration. This seems to be preceeded by the phrase 'session_duration'. Let's check how many assessments contain this phrase.

In [39]:
completed=end.loc[end.type=='Assessment'].event_data.apply(lambda x: x.find('session_duration')>-1)
print('No. of assessments containing the phrase session_duration: {} '.format(completed.loc[completed==True].size))

No. of assessments containing the phrase session_duration: 11691 


This same as the number of session with last event having an event_code 2010. At this point, I am unable to find a generic pattern that fits all completed session. Perhaps, it is wrong to assume that all the sessions in train_labels.csv were completed. It is probably better to look at all sessions which were attempted at least once whether successfully or unsuccessfully and treat them as valid assessments. 

At this point, I have the following 3 dataframes containing various kinds of information extracted from train.csv:

  - start: This contains info about the starting event in each session
  
  - end: This contains info about the last event in each session
  
  - attempts: This contains the events with event_code 4100 or 4110 and corresponds to the players   attempts at solving the assessments


Note that the last event for each session already contains information about its event_count and session_duration i.e. this info is easily available from the dataframe 'end'. To look at the history of a player before they attempt a particular assessment, all I need is to add the session start_time for each session in 'end'. The session start_time can be obtained from the dataframe 'start'. Let's include the session start_time for each session in 'end'.

In [47]:
# join 'end' and 'begin' according to the game_session id
# recall that session_id corresponds to the row index of the dataframes above
# therefore merge should be performed with left_index=True and right_index=True
end=end.merge(start, left_index=True, right_index=True, suffixes=('_end','_begin')).drop(columns=['installation_id_begin'])